In [42]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
import numpy as np

In [43]:
class ModelTrainer:
    def __init__(self, train_dir, val_dir, batch_size=32, img_size=(224, 224)):
        self.train_dir = train_dir
        self.val_dir = val_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.train_generator = None
        self.val_generator = None

    def prepare_data(self):
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True
        )
        val_datagen = ImageDataGenerator(rescale=1./255)

        self.train_generator = train_datagen.flow_from_directory(
            self.train_dir,
            target_size=self.img_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            shuffle=True
        )
        self.val_generator = val_datagen.flow_from_directory(
            self.val_dir,
            target_size=self.img_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            shuffle=True
        )

    def build_model(self, num_classes):
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(*self.img_size, 3)),
            BatchNormalization(),
            MaxPooling2D((2, 2)),

            Conv2D(64, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),

            Conv2D(128, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),

            Conv2D(256, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),

            Conv2D(512, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),

            GlobalAveragePooling2D(),
            Dense(512, activation='relu'),
            BatchNormalization(),
            Dropout(0.5),

            Dense(num_classes, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        self.model = model

    def train_model(self, epochs=100):
        early_stopping = EarlyStopping(
            monitor='val_loss',        
            patience=10,               
            restore_best_weights=True  
        )
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',        
            factor=0.2,                
            patience=5,                
            min_lr=0.0001              
        )

        history = self.model.fit(
            self.train_generator,
            steps_per_epoch=self.train_generator.samples // self.train_generator.batch_size,
            epochs=epochs,
            validation_data=self.val_generator,
            validation_steps=self.val_generator.samples // self.val_generator.batch_size,
            callbacks=[early_stopping, reduce_lr]
        )

        return history

    def save_model(self, model_path):
        self.model.save(model_path)

In [44]:
class ModelEvaluator:
    def __init__(self, model_path, test_dir, batch_size=32, img_size=(224, 224)):
        self.model_path = model_path
        self.test_dir = test_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.test_generator = None

    def load_model(self):
        self.model = tf.keras.models.load_model(self.model_path)

    def prepare_data(self):
        test_datagen = ImageDataGenerator(rescale=1./255)
        self.test_generator = test_datagen.flow_from_directory(
            self.test_dir,
            target_size=self.img_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            shuffle=False
        )

    def evaluate_model(self):
        evaluation = self.model.evaluate(self.test_generator)
        return evaluation

In [56]:
class ImagePredictor:
    def __init__(self, model_path, train_dir, img_size=(224, 224)):
        self.model_path = model_path
        self.train_dir = train_dir
        self.img_size = img_size

    def load_model(self):
        self.model = tf.keras.models.load_model(self.model_path)

    def load_and_preprocess_image(self, img_path):
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=self.img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, axis=0)
        img_array /= 255.0
        return img_array

    def get_class_labels(self):
        self.class_labels = {i: folder for i, folder in enumerate(os.listdir(self.train_dir))}

    def predict_image_class(self, img_path):
        img_array = self.load_and_preprocess_image(img_path)
        predictions = self.model.predict(img_array)
        predicted_class_index = np.argmax(predictions)
        return self.class_labels[predicted_class_index]

In [ ]:
trainer = ModelTrainer(train_dir=r'\train', val_dir=r'\val')
trainer.prepare_data()
trainer.build_model(num_classes=23) 
history = trainer.train_model(epochs=100)
trainer.save_model(r'\nazwa_modelu.h5')

In [46]:
evaluator = ModelEvaluator(model_path=r'\model\mod.h5', test_dir=r'\test')
evaluator.load_model()
evaluator.prepare_data()
evaluation = evaluator.evaluate_model()
print(f"Loss: {evaluation[0]}, Accuracy: {evaluation[1]}")

Found 4752 images belonging to 23 classes.
149/149 ━━━━━━━━━━━━━━━━━━━━ 32s 213ms/step - accuracy: 0.7999 - loss: 0.7007
Loss: 0.754960834980011, Accuracy: 0.7788299918174744


In [61]:
predictor = ImagePredictor(model_path=r'\model\mod.h5', train_dir=r'\train')
predictor.load_model()
predictor.get_class_labels()
predicted_class = predictor.predict_image_class(r'\owca1.jpg')
print(f"Przewidywane zwierzę: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
Przewidywane zwierzę: sheep
